# Option Pricing model

We start with the definition of the various inputs used in the Black-Scholes-Merton model.

Underlying Price (S): The current market price of the asset.

Strike Price (K): The predetermined price at which the option can be exercis



Time to Expiration (T): The time left (in years) until the option’s expiration
te.


Risk Free Rate (r): The constant rate of return on a risk-free asset, such as a governm
Vbond.


Volatility (σ): A measure of how much the price of the underlying asset




$$
C = S N(d_1) - K e^{-rt} N(d_2)
$$

where:

$$
d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)t}{\sigma\sqrt{t}}
$$

and

$$
d_2 = d_1 - \sigma\sqrt{t}
$$

and where:

* $C = \text{Call option price}$
* $S = \text{Current stock (or other underlying) price}$
* $K = \text{Strike price}$
* $r = \text{Risk-free interest rate}$
* $t = \text{Time to maturity}$
* $N = \text{A normal distribution CDF}$
 fluctuates.

In [62]:
import math
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

In [27]:
S = 45  # Underlying Price TBRd
K = 40  # Strike Price TBRd
T = 2   # Time to Maturity TBRd
r = 0.1 # Risk-Free Rate  TBRd
vol = 0.1 # Volatility (σ)  TBRd

# European option pricing

In [49]:
# Calculation of d1 and d2
def BSM_EU (S, K, T, r, vol) :
    
    d1 = (math.log(S/K) + (r + 0.5 * vol **2) * T ) / (vol * math.sqrt(T))
    d2 = d1 - vol*math.sqrt(T)
    
    price_UScall = S * norm.cdf(d1) - K* math.exp(-r*T)*norm.cdf(d2)  
    price_USput =  K* math.exp(-r*T)*norm.cdf(-d2) - S * norm.cdf(-d1)
    
    print (f'The US call option price is {price_UScall:.2f}$.')
    print (f'The US put option price is {price_USput:.2f}$.')

In [51]:
BSM_EU(S,K,T,r,vol)

The US call option price is 12.27$.
The US put option price is 0.02$.


In [59]:
def black_scholes_greeks(S, K, T, r, vol):
    d1 = (math.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * math.sqrt(T))
    d2 = d1 - vol * math.sqrt(T)

    delta_c = norm.cdf(d1)
    gamma_c = norm.pdf(d1) / (S * vol * math.sqrt(T))
    vega_c = S * norm.pdf(d1) * math.sqrt(T) / 100  # per 1% change
    theta_c = (-S * norm.pdf(d1) * vol / (2 * math.sqrt(T)) 
                 - r * K * math.exp(-r * T) * norm.cdf(d2)) / 365
    rho_c = K * T * math.exp(-r * T) * norm.cdf(d2) / 100

    delta_p = -norm.cdf(-d1)
    gamma_p = norm.pdf(d1) / (S * vol * math.sqrt(T))
    vega_p = S * norm.pdf(d1) * math.sqrt(T) / 100  # per 1% change
    thetac_p = (-S * norm.pdf(d1) * vol / (2 * math.sqrt(T)) 
                 + r * K * math.exp(-r * T) * norm.cdf(-d2)) / 365
    rhoc_p = -K * T * math.exp(-r * T) * norm.cdf(-d2) / 100

    print(f"For a European call option:\n"
          f"  Delta: {delta_c:.4f}\n"
          f"  Gamma: {gamma:.4f}\n"
          f"  Vega : {vega:.4f}\n"
          f"  Theta: {theta_c:.4f}\n"
          f"  Rho  : {rho_c:.4f}")

    print(f"\nFor a European put option:\n"
          f"  Delta: {delta_p:.4f}\n"
          f"  Gamma: {gamma:.4f}\n"
          f"  Vega : {vega:.4f}\n"
          f"  Theta: {theta_p:.4f}\n"
          f"  Rho  : {rho_p:.4f}")




# American option

American options differ from European ones in that they can be exercised at any moment before their maturity.

To handle this possibility, we estimated the expected payoff of holding the option until its maturity before arbitrating on whether to keep or sell the option.

In [ ]:
import numpy as np

def bsm_american_put(S0, K, T, r, vol, steps=50, paths=10000):
    dt = T / steps
    discount = np.exp(-r * dt)
    np.random.seed(0)

    # Simulate asset price paths using GBM
    Z = np.random.randn(paths, steps)
    S = np.zeros((paths, steps + 1))
    S[:, 0] = S0

    for t in range(1, steps + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * vol ** 2) * dt + vol * np.sqrt(dt) * Z[:, t - 1])

    # Calculate payoff at all steps (American put)
    payoff = np.maximum(K - S, 0)

    # Start with terminal cashflow
    cashflow = payoff[:, -1]

    # Backward induction for early exercise decision
    for t in range(steps - 1, 0, -1):
        in_the_money = payoff[:, t] > 0
        X = S[in_the_money, t]
        Y = cashflow[in_the_money] * discount

        if len(X) == 0:
            continue

        # Polynomial regression (2nd degree)
        A = np.vstack([np.ones_like(X), X, X**2]).T
        coeffs = np.linalg.lstsq(A, Y, rcond=None)[0]
        continuation = coeffs[0] + coeffs[1] * X + coeffs[2] * X**2

        # Exercise decision
        exercise = payoff[in_the_money, t] > continuation
        cashflow[in_the_money] = np.where(exercise, payoff[in_the_money, t],
                                          cashflow[in_the_money] * discount)

    # Final discounted expected value
    price = np.mean(cashflow) * np.exp(-r * dt)
    return price


Note sur la constuction d'une application

il faudrait un outil de visualisation de grecs sur la période de vie d'une (vraie) option.
-> ça veut dire qu'il faudrait connecter les inputs à un broker